In [ ]:
pip install transformers streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("empathetic_dialogues", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import Dataset
import torch

# Check GPU
print("GPU Available:", torch.cuda.is_available())

# Load dataset
dataset = load_dataset("empathetic_dialogues", split="train[:5%]")  # Use 5% for faster training

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Important for GPT models
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

# Custom dataset class
class EmpatheticDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        utterance = str(self.dataset[idx].get("utterance", ""))
        response = str(self.dataset[idx].get("response", ""))
        sep = " <|sep|> "  # Use custom separator string
        text = utterance + sep + response
        encoding = self.tokenizer(text, return_tensors='pt', truncation=True,
                                  padding='max_length', max_length=self.max_length)
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': encoding['input_ids'].squeeze()
        }

# Prepare dataset
custom_dataset = EmpatheticDataset(dataset, tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Mixed precision for speed
    save_total_limit=1,
    save_strategy="no",  # Disable saving to speed up small runs
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=custom_dataset,
)

# Train the model
trainer.train()


GPU Available: True


Step,Training Loss
10,9.329600
20,8.437900
30,4.038700
40,1.106200
50,0.841000
60,0.673700
70,0.607200
80,0.534700
90,0.527800
100,0.532500


TrainOutput(global_step=120, training_loss=2.3052422126134235, metrics={'train_runtime': 34.3972, 'train_samples_per_second': 111.463, 'train_steps_per_second': 3.489, 'total_flos': 125226467721216.0, 'train_loss': 2.3052422126134235, 'epoch': 1.0})

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_strategy="epoch",  # Save after each epoch
    save_total_limit=1,
    logging_dir="./logs",
    fp16=True,
)


In [ ]:
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/merges.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [ ]:
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
@st.cache_resource
def load_model():
    model = AutoModelForCausalLM.from_pretrained("./results")
    tokenizer = AutoTokenizer.from_pretrained("./results")
    return model, tokenizer

model, tokenizer = load_model()

# Generate a chatbot response
def generate_response(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.95,
        temperature=0.8
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Streamlit UI
def main():
    st.title("🧠 Mental Health Support Chatbot")
    input_text = st.text_area("You:", height=100)
    if st.button("Send") and input_text.strip():
        response = generate_response(input_text)
        st.markdown(f"**Chatbot:** {response}")

if __name__ == "__main__":
    main()


2025-06-22 15:14:37.029 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:14:37.141 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-22 15:14:37.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:14:37.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:14:37.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:14:37.464 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:14:37.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:14:37.480 Thread 'MainThread': mi

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2yrveDhVhpq0rb90gLl4VsR8uHL_4bQg8MmDuAc9T9tBNPjtK")

In [ ]:
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/merges.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [ ]:
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
@st.cache_resource
def load_model():
    model = AutoModelForCausalLM.from_pretrained("./results")
    tokenizer = AutoTokenizer.from_pretrained("./results")
    return model, tokenizer

model, tokenizer = load_model()

# Predefined fallback responses for empathy
fallback_responses = {
    "anxious": "😟 It's okay to feel anxious sometimes. You're not alone. Try some deep breathing — inhale calm, exhale stress.",
    "panic attack": "💗 Panic attacks can be scary. Try to focus on your breathing and remind yourself: this will pass.",
    "lonely": "🤝 Feeling lonely is tough. You're not alone — even sharing your thoughts helps.",
    "overwhelmed": "🌀 When you're overwhelmed, take one small step. Then another. That's progress.",
    "unmotivated": "🚶 Start small — even brushing your teeth or drinking water counts. You're doing your best.",
    "not good enough": "💙 You are enough. Just as you are.",
    "empty": "💬 It's okay to feel numb. You don't have to pretend. You're allowed to feel.",
    "rejection": "🛡 Rejection hurts, but it doesn’t define your worth. You are still valuable.",
    "fight with friend": "💬 Arguments happen. When you're ready, honest communication can heal a lot.",
    "toxic people": "🧱 Boundaries protect your peace. It's okay to choose distance.",
    "encouragement": "🌟 You’ve come so far. Be proud of yourself. You matter.",
    "calming": "🌿 Take a moment. Breathe in... and out. You’re safe right now."
}

# Response generation with fallback
def generate_response(input_text):
    input_lower = input_text.lower()
    for keyword, reply in fallback_responses.items():
        if keyword in input_lower:
            return reply

    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.95,
        temperature=0.8
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 🌟 Streamlit App UI
def main():
    # Set background color using HTML/CSS
    st.markdown("""
        <style>
            .main {
                background-color: #f0f2f6;
                padding: 20px;
                border-radius: 10px;
            }
            .user-bubble {
                background-color: #d0e8ff;
                padding: 10px;
                border-radius: 10px;
                margin: 10px 0;
            }
            .bot-bubble {
                background-color: #e2ffe6;
                padding: 10px;
                border-radius: 10px;
                margin: 10px 0;
            }
        </style>
    """, unsafe_allow_html=True)

    st.title("🧠 Mental Health Support Chatbot")
    st.markdown("Feeling something? Just share. This space is for you. 🤗")

    input_text = st.text_area("You:", height=100)

    if st.button("Send") and input_text.strip():
        st.markdown(f"<div class='user-bubble'>🙋‍♀️ {input_text}</div>", unsafe_allow_html=True)
        response = generate_response(input_text)
        st.markdown(f"<div class='bot-bubble'>🤖 {response}</div>", unsafe_allow_html=True)

if __name__ == "__main__":
    main()


2025-06-22 15:31:58.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:31:58.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:31:58.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:31:58.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:31:58.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:31:58.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:31:58.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 15:31:58.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
# Install required packages
!pip install -q streamlit transformers pyngrok

# Set up ngrok with your token
from pyngrok import ngrok
ngrok.set_auth_token("2yrveDhVhpq0rb90gLl4VsR8uHL_4bQg8MmDuAc9T9tBNPjtK")

# Save the chatbot Streamlit app to a file
chatbot_code = """
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

@st.cache_resource
def load_model():
    model = AutoModelForCausalLM.from_pretrained("./results")
    tokenizer = AutoTokenizer.from_pretrained("./results")
    return model, tokenizer

model, tokenizer = load_model()

def generate_response(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.95,
        temperature=0.8
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def main():
    st.title("🧠 Mental Health Support Chatbot")
    input_text = st.text_area("You:", height=100)
    if st.button("Send") and input_text.strip():
        response = generate_response(input_text)
        st.markdown(f"**Chatbot:** {response}")

if __name__ == "__main__":
    main()
"""

# Save the app code to a Python file
with open("chatbot_app.py", "w") as f:
    f.write(chatbot_code)

# Start Streamlit using ngrok
import os
import threading

public_url = ngrok.connect(addr=8501)
print(f"🚀 Your chatbot is running at: {public_url}")

def run():
    os.system("streamlit run chatbot_app.py")

threading.Thread(target=run).start()


🚀 Your chatbot is running at: NgrokTunnel: "https://e401-34-87-172-181.ngrok-free.app" -> "http://localhost:8501"
